## Agregação por distrito

Nesse notebook é feita a agregação dos dados por distritos do município

In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import LineString
from utils.load_csv import load_csv
from utils.load_shp import load_shp

In [2]:
df_final = load_csv("df_final.csv")

distritos = load_shp("distritos.shp")

In [3]:
df_final["lon"] = df_final["ponto_inicial"].apply(lambda x: eval(x)[0])
df_final["lat"] = df_final["ponto_inicial"].apply(lambda x: eval(x)[1])

In [4]:
gdf = gpd.GeoDataFrame(df_final, geometry=gpd.points_from_xy(df_final.lon, df_final.lat), crs="EPSG:4326")

In [5]:
trajetos = gdf.groupby("id_onibus")["geometry"].apply(lambda x: LineString(x.tolist()))
trajetos = gpd.GeoDataFrame(trajetos, geometry="geometry", crs="EPSG:4326").reset_index()

In [6]:
trajetos = trajetos.to_crs(distritos.crs)

In [7]:
trajetos_distritos = gpd.sjoin(trajetos, distritos, how="inner", predicate="intersects")

In [8]:
# ônibus por distrito
onibus_por_distrito = trajetos_distritos.groupby("nm_distrit")["id_onibus"].nunique().reset_index()

# distância percorrida por distrito
distancia_por_onibus = df_final.groupby("id_onibus")["distancia_percorrida"].sum().reset_index()
trajetos_distritos = trajetos_distritos.merge(distancia_por_onibus, on="id_onibus")
dist_por_distrito = trajetos_distritos.groupby("nm_distrit")["distancia_percorrida"].sum().reset_index()